In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#url = 'https://raw.githubusercontent.com/MariaRosendal/Enhancing-Price-Momentum-with-RNN/main/'
url= ""

## 1. Importing input and output variables

In [70]:
# Import X 
X_train = pickle.load(open('X_train_scaled.pkl', 'rb' ))
X_test = pickle.load(open('X_test_scaled.pkl', 'rb' ))

In [203]:
# Checking for NaN values
print(np.count_nonzero(np.isnan(X_train)), np.count_nonzero(np.isnan(X_test)))

0 0


In [204]:
X_train.shape, X_test.shape, 

((270000, 12, 9), (270000, 12, 9))

In [205]:
# Importing y
y = pd.read_csv(url+'y_reg.csv')
y['date'] =  pd.to_datetime(y['date'])
y = y.set_index(['date'])

In [207]:
# Defining length of y_train and y_test
y_train = y[y.index <= '1975-12-01']
y_test = y[y.index > '1975-12-01']

In [208]:
y_train.shape, y_test.shape

((270000, 1), (270000, 1))

In [209]:
# Reformating y
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_train.shape, y_test.shape

((270000,), (270000,))

## 2. Models

In [210]:
# Importing packages
import tensorflow as tf
import matplotlib.pyplot as plt

import math
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import BatchNormalization 
from keras.layers.core import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import r2_score

### MLP

In [81]:
# Reshaping X to vectors
n_input = X_train.shape[1] * X_train.shape[2]
X_train_mlp = X_train.reshape(len(X_train),n_input)
X_test_mlp = X_test.reshape(len(X_test),n_input)

In [87]:
X_train_mlp.shape,  X_test_mlp.shape

((270000, 108), (270000, 108))

In [88]:
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [90]:
epochs = [50, 500] 
batch_size = 500
lr = [1e-5, 1e-3]
neurons = [5, 100]
dropout = [0, 0.7]

In [91]:
# MLP model
def create_mlp(epochs=epochs, batch_size=batch_size, lr=lr, neurons=neurons, dropout=dropout):
    model_mlp = Sequential()
    model_mlp.add(Dense(neurons, input_dim=n_input)) # 120, 80, 50, 1
    model_mlp.add(Dropout(dropout))       
    model_mlp.add(BatchNormalization())
    model_mlp.add(Dense(neurons//2, activation='relu'))
    model_mlp.add(Dropout(dropout))
    model_mlp.add(BatchNormalization())
    model_mlp.add(Dense(neurons//4, activation='relu'))
    model_mlp.add(Dense(1))
    optimizer=keras.optimizers.Adam(lr=lr)
    #Compile model
    model_mlp.compile(loss='mse', optimizer=optimizer) 
    return model_mlp

In [92]:
# Implement early stopping
def on_step(optim_result):
    EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10, min_delta=0.1)
    return True

In [93]:
# Wrapper to run Keras in ScikitLearn
mlp_opt = KerasRegressor(build_fn=create_mlp)

In [94]:
param_opt = dict(epochs=Integer(50, 500), neurons=Integer(5,100), dropout=Real(0, 0.7), lr = Real(1e-5, 1e-3))

In [95]:
# Ensuring data keeps temporal element when split
tscv = TimeSeriesSplit(n_splits=9) 

In [96]:
bayesian = BayesSearchCV(
     mlp_opt,
     param_opt,
     n_iter=32,
     random_state=500,
     cv=tscv,
     verbose = 0, n_jobs=1
 )

bayesian_result = bayesian.fit(X_train_mlp, y_train, callback=on_step)

Epoch 1/291
844/844 [==============================] - 2s 1ms/step - loss: 0.2397
Epoch 2/291
844/844 [==============================] - 1s 1ms/step - loss: 0.0136
Epoch 3/291
844/844 [==============================] - 1s 1ms/step - loss: 0.0042
Epoch 4/291
844/844 [==============================] - 1s 1ms/step - loss: 0.0016
Epoch 5/291
844/844 [==============================] - 1s 1ms/step - loss: 6.8535e-04
Epoch 6/291
844/844 [==============================] - 1s 1ms/step - loss: 2.9480e-04
Epoch 7/291
844/844 [==============================] - 1s 1ms/step - loss: 1.2526e-04
Epoch 8/291
844/844 [==============================] - 1s 1ms/step - loss: 4.8279e-05
Epoch 9/291
844/844 [==============================] - 1s 1ms/step - loss: 1.7743e-05
Epoch 10/291
844/844 [==============================] - 1s 1ms/step - loss: 8.4314e-06
Epoch 11/291
844/844 [==============================] - 1s 1ms/step - loss: 6.7057e-06
Epoch 12/291
844/844 [==============================] - 1s 1ms/step 

844/844 [==============================] - 1s 1ms/step - loss: 4.2150e-06
Epoch 96/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2465e-06
Epoch 97/291
844/844 [==============================] - 1s 1ms/step - loss: 4.4311e-06
Epoch 98/291
844/844 [==============================] - 1s 1ms/step - loss: 4.4526e-06
Epoch 99/291
844/844 [==============================] - 1s 1ms/step - loss: 4.4508e-06
Epoch 100/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2387e-06
Epoch 101/291
844/844 [==============================] - 1s 1ms/step - loss: 4.6343e-06
Epoch 102/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2668e-06
Epoch 103/291
844/844 [==============================] - 1s 1ms/step - loss: 4.6259e-06
Epoch 104/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3878e-06
Epoch 105/291
844/844 [==============================] - 1s 1ms/step - loss: 4.4103e-06
Epoch 106/291
844/844 [===========================

844/844 [==============================] - 1s 1ms/step - loss: 4.2495e-06
Epoch 189/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3277e-06
Epoch 190/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1841e-06
Epoch 191/291
844/844 [==============================] - 1s 1ms/step - loss: 4.0952e-06
Epoch 192/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2252e-06
Epoch 193/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3275e-06
Epoch 194/291
844/844 [==============================] - 1s 1ms/step - loss: 4.0977e-06
Epoch 195/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1826e-06
Epoch 196/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3128e-06
Epoch 197/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1380e-06
Epoch 198/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1856e-06
Epoch 199/291
844/844 [=======================

844/844 [==============================] - 1s 1ms/step - loss: 4.3140e-06
Epoch 281/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1343e-06
Epoch 282/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2649e-06
Epoch 283/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3914e-06
Epoch 284/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1526e-06
Epoch 285/291
844/844 [==============================] - 1s 1ms/step - loss: 4.3660e-06
Epoch 286/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1413e-06
Epoch 287/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1970e-06
Epoch 288/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2257e-06
Epoch 289/291
844/844 [==============================] - 1s 1ms/step - loss: 4.1385e-06
Epoch 290/291
844/844 [==============================] - 1s 1ms/step - loss: 4.2862e-06
Epoch 291/291
844/844 [=======================

1688/1688 [==============================] - 2s 1ms/step - loss: 3.5689e-06
Epoch 81/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5934e-06
Epoch 82/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5458e-06
Epoch 83/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5334e-06
Epoch 84/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.6216e-06
Epoch 85/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5243e-06
Epoch 86/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5133e-06
Epoch 87/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.4489e-06
Epoch 88/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.4490e-06
Epoch 89/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5033e-06
Epoch 90/291
1688/1688 [==============================] - 2s 1ms/step - loss: 3.5591e-06
Epoch 91/291
1688/1688 [==========

...

7594/7594 [==============================] - 7s 880us/step - loss: 1.4623e-06
Epoch 202/291
7594/7594 [==============================] - 7s 873us/step - loss: 1.4475e-06
Epoch 203/291
7594/7594 [==============================] - 7s 872us/step - loss: 1.4783e-06
Epoch 204/291
7594/7594 [==============================] - 7s 868us/step - loss: 1.4628e-06
Epoch 205/291
7594/7594 [==============================] - 7s 871us/step - loss: 1.4344e-06
Epoch 206/291
7594/7594 [==============================] - 7s 873us/step - loss: 1.4314e-06
Epoch 207/291
7594/7594 [==============================] - 7s 882us/step - loss: 1.4451e-06
Epoch 208/291
7594/7594 [==============================] - 7s 880us/step - loss: 1.4596e-06
Epoch 209/291
7594/7594 [==============================] - 7s 876us/step - loss: 1.4647e-06
Epoch 210/291
7594/7594 [==============================] - 7s 867us/step - loss: 1.4276e-06
Epoch 211/291
7594/7594 [==============================] - 7s 877us/step - loss: 1.4774e-06
Ep

7594/7594 [==============================] - 7s 877us/step - loss: 1.4377e-06
Epoch 291/291
844/844 [==============================] - 0s 406us/step - loss: 1.1638e-06
Epoch 1/291
8438/8438 [==============================] - 8s 867us/step - loss: 0.0557
Epoch 2/291
8438/8438 [==============================] - 7s 803us/step - loss: 5.2185e-06
Epoch 3/291
8438/8438 [==============================] - 7s 773us/step - loss: 1.7145e-06
Epoch 4/291
8438/8438 [==============================] - 6s 764us/step - loss: 1.4431e-06
Epoch 5/291
8438/8438 [==============================] - 6s 765us/step - loss: 1.4155e-06
Epoch 6/291
8438/8438 [==============================] - 6s 741us/step - loss: 1.4183e-06
Epoch 7/291
8438/8438 [==============================] - 6s 723us/step - loss: 1.4214e-06
Epoch 8/291
8438/8438 [==============================] - 6s 716us/step - loss: 1.4320e-06
Epoch 9/291
8438/8438 [==============================] - 6s 717us/step - loss: 1.4086e-06
Epoch 10/291
8438/8438 [==

8438/8438 [==============================] - 6s 714us/step - loss: 1.4384e-06
Epoch 89/291
8438/8438 [==============================] - 6s 707us/step - loss: 1.4138e-06
Epoch 90/291
8438/8438 [==============================] - 6s 708us/step - loss: 1.4073e-06
Epoch 91/291
8438/8438 [==============================] - 6s 711us/step - loss: 1.4017e-06
Epoch 92/291
8438/8438 [==============================] - 6s 706us/step - loss: 1.3968e-06
Epoch 93/291
8438/8438 [==============================] - 6s 715us/step - loss: 1.4096e-06
Epoch 94/291
8438/8438 [==============================] - 6s 716us/step - loss: 1.3911e-06
Epoch 95/291
8438/8438 [==============================] - 6s 714us/step - loss: 1.4172e-06
Epoch 96/291
8438/8438 [==============================] - 6s 717us/step - loss: 1.4041e-06
Epoch 97/291
8438/8438 [==============================] - 6s 714us/step - loss: 1.4128e-06
Epoch 98/291
8438/8438 [==============================] - 6s 713us/step - loss: 1.4260e-06
Epoch 99/291

8438/8438 [==============================] - 6s 709us/step - loss: 1.4280e-06
Epoch 178/291
8438/8438 [==============================] - 6s 704us/step - loss: 1.3976e-06
Epoch 179/291
8438/8438 [==============================] - 6s 709us/step - loss: 1.3968e-06
Epoch 180/291
8438/8438 [==============================] - 6s 707us/step - loss: 1.4154e-06
Epoch 181/291
8438/8438 [==============================] - 6s 714us/step - loss: 1.4066e-06
Epoch 182/291
8438/8438 [==============================] - 6s 714us/step - loss: 1.4055e-06
Epoch 183/291
8438/8438 [==============================] - 6s 717us/step - loss: 1.4179e-06
Epoch 184/291
8438/8438 [==============================] - 6s 719us/step - loss: 1.4067e-06
Epoch 185/291
8438/8438 [==============================] - 6s 717us/step - loss: 1.4180e-06
Epoch 186/291
8438/8438 [==============================] - 6s 719us/step - loss: 1.4496e-06
Epoch 187/291
8438/8438 [==============================] - 6s 718us/step - loss: 1.4009e-06
Ep

8438/8438 [==============================] - 6s 715us/step - loss: 1.4169e-06
Epoch 267/291
8438/8438 [==============================] - 6s 708us/step - loss: 1.4178e-06
Epoch 268/291
8438/8438 [==============================] - 6s 704us/step - loss: 1.4155e-06
Epoch 269/291
8438/8438 [==============================] - 6s 710us/step - loss: 1.3967e-06
Epoch 270/291
8438/8438 [==============================] - 6s 710us/step - loss: 1.4125e-06
Epoch 271/291
8438/8438 [==============================] - 6s 709us/step - loss: 1.3919e-06
Epoch 272/291
8438/8438 [==============================] - 6s 711us/step - loss: 1.4044e-06
Epoch 273/291
8438/8438 [==============================] - 6s 706us/step - loss: 1.4178e-06
Epoch 274/291
8438/8438 [==============================] - 6s 710us/step - loss: 1.4380e-06
Epoch 275/291
8438/8438 [==============================] - 6s 713us/step - loss: 1.4173e-06
Epoch 276/291
8438/8438 [==============================] - 6s 708us/step - loss: 1.4064e-06
Ep

In [97]:
# Best model parameters
print('According to BayesianOptimization the best parameters are : ')
#print('batch_size : ' + str(bayesian_result.best_params_['batch_size']))
print('epochs : ' + str(bayesian_result.best_params_['epochs']))
print('neurons : ' + str(bayesian_result.best_params_['neurons']))
print('dropout : ' + str(bayesian_result.best_params_['dropout']))
print('lr : ' + str(bayesian_result.best_params_['lr']))

According to BayesianOptimization the best parameters are : 
epochs : 291
neurons : 59
dropout : 0.6867449131712161
lr : 0.0006170997584834369


In [144]:
# Selecting best model
print("Best: %f using %s" % (bayesian_result.best_score_, bayesian_result.best_params_))
means = bayesian_result.cv_results_['mean_test_score']
stds = bayesian_result.cv_results_['std_test_score']
params = bayesian_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.000002 using OrderedDict([('dropout', 0.6867449131712161), ('epochs', 291), ('lr', 0.0006170997584834369), ('neurons', 59)])
-0.000002 (0.000002) with: OrderedDict([('dropout', 0.6867449131712161), ('epochs', 291), ('lr', 0.0006170997584834369), ('neurons', 59)])


In [244]:
# Defining best model parameters
#batch_size = bayesian_result.best_params_['batch_size']
epochs = bayesian_result.best_params_['epochs']
neurons = bayesian_result.best_params_['neurons']
dropout = bayesian_result.best_params_['dropout']
lr = bayesian_result.best_params_['lr']

In [245]:
# Defining optimized model
opt_mlp = Sequential()
opt_mlp.add(Dense(neurons, input_dim=n_input))
opt_mlp.add(Dropout(dropout))
opt_mlp.add(BatchNormalization())
opt_mlp.add(Dense(neurons//2, activation='relu'))
opt_mlp.add(Dropout(dropout))
opt_mlp.add(BatchNormalization())
opt_mlp.add(Dense(neurons//4, activation='relu'))
opt_mlp.add(Dense(1))
optimizer=keras.optimizers.Adam(lr=0.00006)
# Compile model
opt_mlp.compile(loss='mse', optimizer=optimizer) 

In [246]:
opt_mlp.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 59)                6431      
_________________________________________________________________
dropout_40 (Dropout)         (None, 59)                0         
_________________________________________________________________
batch_normalization_38 (Batc (None, 59)                236       
_________________________________________________________________
dense_81 (Dense)             (None, 29)                1740      
_________________________________________________________________
dropout_41 (Dropout)         (None, 29)                0         
_________________________________________________________________
batch_normalization_39 (Batc (None, 29)                116       
_________________________________________________________________
dense_82 (Dense)             (None, 14)              

In [247]:
#history = opt_mlp.fit(X_train_mlp, y_train, epochs=200, batch_size=100, validation_split=0.2)
history = opt_mlp.fit(X_train_mlp, y_train, epochs=epochs, batch_size=batch_size, shuffle='batch')

Epoch 1/291
540/540 [==============================] - 1s 1ms/step - loss: 0.8046
Epoch 2/291
540/540 [==============================] - 1s 1ms/step - loss: 0.2883
Epoch 3/291
540/540 [==============================] - 1s 1ms/step - loss: 0.1470
Epoch 4/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0948
Epoch 5/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0681
Epoch 6/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0531
Epoch 7/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0415
Epoch 8/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0330
Epoch 9/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0262
Epoch 10/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0201
Epoch 11/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0158
Epoch 12/291
540/540 [==============================] - 1s 1ms/step - loss: 0.0123
Epoch 13/291


540/540 [==============================] - 1s 1ms/step - loss: 1.4391e-06
Epoch 97/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4525e-06
Epoch 98/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4249e-06
Epoch 99/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4120e-06
Epoch 100/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4008e-06
Epoch 101/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4108e-06
Epoch 102/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4174e-06
Epoch 103/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4386e-06
Epoch 104/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4268e-06
Epoch 105/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4213e-06
Epoch 106/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4320e-06
Epoch 107/291
540/540 [==========================

540/540 [==============================] - 1s 1ms/step - loss: 1.3974e-06
Epoch 190/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4282e-06
Epoch 191/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4343e-06
Epoch 192/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3917e-06
Epoch 193/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3954e-06
Epoch 194/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4082e-06
Epoch 195/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4229e-06
Epoch 196/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4447e-06
Epoch 197/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4048e-06
Epoch 198/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4013e-06
Epoch 199/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3969e-06
Epoch 200/291
540/540 [=======================

540/540 [==============================] - 1s 1ms/step - loss: 1.3702e-06
Epoch 283/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3537e-06
Epoch 284/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3913e-06
Epoch 285/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4110e-06
Epoch 286/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3943e-06
Epoch 287/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3848e-06
Epoch 288/291
540/540 [==============================] - 1s 1ms/step - loss: 1.4151e-06
Epoch 289/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3703e-06
Epoch 290/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3800e-06
Epoch 291/291
540/540 [==============================] - 1s 1ms/step - loss: 1.3886e-06


In [248]:
mlp_predict = opt_mlp.predict(X_test_mlp, verbose=0)
#mlp_predict = model_mlp.predict(X_test_mlp, verbose=0)

In [249]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = mlp_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])

In [250]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = mlp_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])
r2_df['y_2'] = r2_df['y']**2
r2_df['dif2'] = r2_df['difference']**2
y_2 = r2_df['y_2'].sum() 
dif2 = r2_df['dif2'].sum()
1 - (dif2 / y_2)

0.005726999868631233

In [251]:
1 - (dif2 / y_2)

0.005726999868631233

In [252]:
mlp = pd.DataFrame(mlp_predict)

mlp.nunique()

0    34744
dtype: int64

In [253]:
mlp.shape

(270000, 1)

In [255]:
pickle.dump(mlp_predict, open('mlp_predict_5.pkl', 'wb' ))